# Actividad 3 | Aprendizaje supervisado y no supervisado

Curso: Análisis de grandes volúmenes de datos

Alumno: Luis Daniel Ortega Muñoz | A01795197

In [1]:
import kagglehub
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, hour, date_format, count, round, concat_ws

C:\Users\danie\anaconda3\envs\env_pyspark\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Introducción teórica


## Selección de los datos

### Carga del conjunto de datos con PySpark

In [2]:
# Download the latest version
path = kagglehub.dataset_download("sobhanmoosavi/us-accidents")

print("Path to dataset files:", path)

dataset_path = path + "/US_Accidents_March23.csv"

Path to dataset files: C:\Users\danie\.cache\kagglehub\datasets\sobhanmoosavi\us-accidents\versions\13


In [3]:
# Create a Spark session
spark = SparkSession.builder \
    .appName("USTrafficAccidents_ML") \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark

In [4]:
df = spark.read.csv(dataset_path, header=True, inferSchema=True)

df.show(5)

+---+-------+--------+-------------------+-------------------+-----------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
| ID| Source|Severity|         Start_Time|           End_Time|        Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Ameni

### Sub-muestreo del conjunto de datos

In [5]:
df = df.withColumn("Weather_Type",
    when(col("Weather_Condition").isNull(), "Desconocido")
    .when(col("Weather_Condition").rlike("(?i)null|N/A"), "Desconocido")
    .when(col("Weather_Condition").rlike("(?i)Rain|Drizzle|Thunder|Storm|Snow|Sleet|Hail|Ice|Fog|Haze|Mist|Dust|Sand|Smoke|Wintry|Squall|Tornado|Ash|Funnel"), "Adverso")
    .otherwise("Favorable")
)
df.groupBy("Weather_Type").count().withColumn("porcentaje", col("count") / df.count() * 100).show()

+------------+-------+------------------+
|Weather_Type|  count|        porcentaje|
+------------+-------+------------------+
| Desconocido| 176711|2.2865164483073714|
|     Adverso| 981225|12.696363565315123|
|   Favorable|6570458| 85.01711998637751|
+------------+-------+------------------+



In [6]:
df = df.withColumn(
    "Hora_Periodo",
    when(hour("Start_Time") < 6, "Madrugada")
    .when(hour("Start_Time") < 18, "Alta actividad")
    .otherwise("Tarde-Noche")
)

df.groupBy("Hora_Periodo").count().withColumn("porcentaje", col("count") / df.count() * 100).show()

+--------------+-------+------------------+
|  Hora_Periodo|  count|        porcentaje|
+--------------+-------+------------------+
|   Tarde-Noche|1438025|18.607035303841908|
|Alta actividad|5515796| 71.37053312758123|
|     Madrugada| 774573| 10.02243156857686|
+--------------+-------+------------------+



In [7]:
# Paso 1: Extraer el día de la semana en formato abreviado (Mon, Tue, ..., Sun)
df = df.withColumn("Dia_Semana", date_format("Start_Time", "E"))

# Paso 2: Clasificar como "Laboral" o "Fin de semana"
df = df.withColumn(
    "Tipo_Día",
    when(col("Dia_Semana").isin("Sat", "Sun"), "Fin de semana").otherwise("Laboral")
)

# # Paso 3: Calcular distribución porcentual
# df.groupBy("Tipo_Día") \
#   .count() \
#   .withColumn("porcentaje", col("count") / df.count() * 100) \
#   .show()

In [8]:
# # Revisar ocurrencia de variables booleanas
# variables_viales = ["Junction", "Crossing", "Traffic_Signal", "Amenity", "Bump", "Give_Way", "No_Exit", "Railway", "Roundabout", "Station", "Stop", "Traffic_Calming", "Turning_Loop"]
#
# for var in variables_viales:
#     df.groupBy(var) \
#       .count() \
#       .withColumn("porcentaje", col("count") / df.count() * 100) \
#       .show()

In [9]:
df_filtrado = df.filter(
    (col("Severity").isNotNull()) &
    (col("Hora_Periodo").isNotNull()) &
    (col("Tipo_Día").isNotNull()) &
    (col("Weather_Type").isNotNull())
)

# Paso 2: Contar el total de registros filtrados
total_registros = df_filtrado.count()

# Paso 3: Agrupar por combinación de estratos y calcular frecuencia y probabilidad empírica
estratos = df_filtrado.groupBy("Severity", "Hora_Periodo", "Tipo_Día", "Weather_Type") \
    .agg(count("*").alias("frecuencia")) \
    .withColumn("probabilidad", round(col("frecuencia") / total_registros, 6)) \
    .orderBy(col("probabilidad").desc())

In [10]:
# Tamaño total de muestra deseado
tamanio_muestra_total = 10000

# Calcular n_estrato para cada combinación
estratos = estratos.withColumn(
    "n_estrato",
    round(col("probabilidad") * tamanio_muestra_total).cast("integer")
)

In [11]:
# En df_filtrado (base depurada sin nulos en variables clave)
df_filtrado = df_filtrado.withColumn(
    "estrato_id",
    concat_ws("_", "Severity", "Hora_Periodo", "Tipo_Día", "Weather_Type")
)

# Igual en la tabla de estratos con probabilidades y n_estrato
estratos = estratos.withColumn(
    "estrato_id",
    concat_ws("_", "Severity", "Hora_Periodo", "Tipo_Día", "Weather_Type")
)

In [12]:
df_muestreo = df_filtrado.join(
    estratos.select("estrato_id", "n_estrato"),
    on="estrato_id",
    how="inner"
)

In [13]:
from pyspark.sql.functions import rand
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Asignar un número aleatorio y calcular el orden por estrato
df_muestreo = df_muestreo.withColumn("rand", rand(seed=42))

window = Window.partitionBy("estrato_id").orderBy("rand")

df_muestreo = df_muestreo.withColumn("row_num", row_number().over(window))

In [14]:
df_muestra_final = df_muestreo.filter(col("row_num") <= col("n_estrato"))
df_muestra_final.summary()

summary,estrato_id,ID,Source,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Weather_Type,Hora_Periodo,Dia_Semana,Tipo_Día,n_estrato,rand,row_num
count,10003,10003,10003,10003,10003,10003,5595,5595,10003,10003,9992,10003,10003,10003,10001,10003,9999,9976,9790,7470,9776,9823,9774,9771,9323,7202,9774,9978,9978,9978,9978,10003,10003,10003,10003,10003,10003,10003
mean,NULL,NULL,NULL,2.212636209137259,36.24469909840036,-94.61993688756752,36.34681220541541,-95.7346700438495,0.5559247225484412,NULL,NULL,NULL,NULL,NULL,56379.03567874911,NULL,NULL,NULL,61.71598569969356,58.311485943775125,64.96389116202946,29.53263870507992,9.099752404338043,NULL,7.593403410919228,0.007060538739239098,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2107.662801159652,6.466151770978525E-4,1054.331400579826
stddev,NULL,NULL,NULL,0.48787772380159755,5.070605427026113,17.38554958180918,5.263693355873905,18.136207202853882,1.597624680398425,NULL,NULL,NULL,NULL,NULL,31097.037297564257,NULL,NULL,NULL,18.9999059851054,22.32139247726831,22.82312340594708,1.0044580229406097,2.5724033627643097,NULL,5.192230141415713,0.041519987983880714,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1861.2403085753288,3.789342770944724E-4,1234.8936379224372
min,1_Alta actividad_...,A-1000027,Source1,1,24.954442999999998,-124.486977,25.451418,-124.486179,0.0,#1 #2 #3 lane blo...,1/2 Rd,Abbeville,Abbeville,AL,01373-9764,US,US/Central,K04W,-15.0,-31.0,2.0,19.82,0.0,CALM,0.0,0.0,Blowing Snow,Day,Day,Day,Day,Adverso,Alta actividad,Fri,Fin de semana,1,3.88564060815888E-7,1
25%,NULL,NULL,NULL,2,33.427165,-117.217077,33.547271,-117.809275,0.0,NULL,NULL,NULL,NULL,NULL,29334.0,NULL,NULL,NULL,49.0,43.0,49.0,29.36,10.0,NULL,4.6,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,501,3.258897606087663E-4,119
50%,NULL,NULL,NULL,2,35.87199000000001,-87.630844,36.54587,-87.92061,0.034,NULL,NULL,NULL,NULL,NULL,55413.0,NULL,NULL,NULL,64.0,62.0,67.0,29.85,10.0,NULL,7.0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,995,6.423893092273314E-4,473
75%,NULL,NULL,NULL,2,40.106846000000004,-80.315125,40.25729854,-80.2061,0.489,NULL,NULL,NULL,NULL,NULL,90650.0,NULL,NULL,NULL,76.0,75.0,84.0,30.03,10.0,NULL,10.0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,4251,9.60305785783433E-4,1751
max,4_Tarde-Noche_Lab...,A-99962,Source3,4,48.930842,-69.907569,48.93099,-69.884072,37.27999877929688,sb 29 jso 11th. 7...,Zorn Ave,Zumbrota,Yuma,WY,99338,US,US/Pacific,KZZV,113.0,111.0,100.0,30.87,60.0,West,33.0,0.99,Wintry Mix,Night,Night,Night,Night,Favorable,Tarde-Noche,Wed,Laboral,4251,0.005325347834556515,4251


## Preparación de los datos

## Preparación del conjunto de entrenamiento y prueba

## Construcción de modelos de aprendizaje supervisado y no supervisado

### Aprendizaje supervisado: Regresión lineal

### Aprendizaje no supervisado: K-Means